# 1C Customers dynamic features


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
import pandas as pd

sys.path.append("..")

from src.utils.core_utils import setup_logging

# Initialize logging
file_log = "customer_dynamic_feature.log"
root_logger = setup_logging(level=logging.DEBUG, log_file=file_log, remove_existing=True)

2025-06-03 11:51:14,660 - src.utils.core_utils - INFO - Creating file handler with level: 10
2025-06-03 11:51:14,661 - src.utils.core_utils - DEBUG - Logging setup complete to customer_dynamic_feature.log


In [3]:
from src.feature_customers import CustomerDynamicFeaturePipelineConfig, CustomerDynamicFeaturePipeline

/Users/sharonshiao/Dropbox/Machine_learning/projects/fahsion_recommendation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run pipeline


In [4]:
customer_dynamic_feature_config = CustomerDynamicFeaturePipelineConfig.create_default()

In [5]:
customer_dynamic_feature_pipeline = CustomerDynamicFeaturePipeline(customer_dynamic_feature_config)

In [6]:
customer_dynamic_feature_pipeline.setup()

2025-06-03 11:51:17,456 - src.feature_customers - INFO - Setting up CustomerDynamicFeaturePipeline
2025-06-03 11:51:17,457 - src.feature_customers - DEBUG - Config: {
  "config_processor": {
    "history_start_week_num": 52,
    "history_end_week_num": 104,
    "start_week_num": 95,
    "end_week_num": 104,
    "k_items": 5
  },
  "subsample": 0.05,
  "seed": 42
}


In [7]:
results_customer_dynamic_feature = customer_dynamic_feature_pipeline.run()

2025-06-03 11:51:17,476 - src.feature_customers - INFO - Loading data for CustomerDynamicFeaturePipeline
2025-06-03 11:51:17,477 - src.feature_extraction - INFO - Loading optimized raw data from transactions train 0.05 42
2025-06-03 11:51:17,477 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_train_sample_0.05_42.parquet
2025-06-03 11:51:17,610 - src.feature_extraction - INFO - Loading optimized raw data from transactions valid 0.05 42
2025-06-03 11:51:17,611 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_valid_sample_0.05_42.parquet
2025-06-03 11:51:17,614 - src.feature_extraction - INFO - Loading optimized raw data from transactions test 0.05 42
2025-06-03 11:51:17,614 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_test_sample_0.05_42.parquet
2025-06-03 11:51:17,631 - src.features_articles - INFO - Loading article embedding result from ../data/preprocessed/articles_embedding/full
2025-06-03 11:51:17,703 -

# Checks


In [8]:
results_customer_dynamic_feature.data.info()
print(results_customer_dynamic_feature.data.shape)
display(results_customer_dynamic_feature.data.head())
print(results_customer_dynamic_feature.feature_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218330 entries, 0 to 218329
Data columns (total 5 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   customer_id                   218330 non-null  uint64 
 1   week_num                      218330 non-null  int64  
 2   customer_avg_price            218330 non-null  float64
 3   customer_avg_text_embedding   200903 non-null  object 
 4   customer_avg_image_embedding  200903 non-null  object 
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 8.3+ MB
(218330, 5)


,customer_id,week_num,customer_avg_price,customer_avg_text_embedding,customer_avg_image_embedding
0,1296218836199721,95,0.026078,NaN,NaN
1,1296218836199721,96,0.025866,NaN,NaN
2,1296218836199721,97,0.025634,NaN,NaN
3,1296218836199721,98,0.025449,NaN,NaN
4,1296218836199721,99,0.025439,NaN,NaN


{'numerical_features': ['customer_avg_price'], 'embedding_features': ['customer_avg_text_embedding', 'customer_avg_image_embedding'], 'one_hot_features': [], 'categorical_features': [], 'id_columns': ['customer_id', 'week_num']}


In [9]:
# Check the shape of the dataframe should match number of customers * number of weeks
print(results_customer_dynamic_feature.data.shape)
print(results_customer_dynamic_feature.data.customer_id.nunique())
print(results_customer_dynamic_feature.data.week_num.value_counts())

(218330, 5)
21833
week_num
95     21833
96     21833
97     21833
98     21833
99     21833
100    21833
101    21833
102    21833
103    21833
104    21833
Name: count, dtype: int64


## Check that the avg embeddings make sense


In [10]:
import numpy as np

from src.feature_customers import CustomerDynamicFeatureProcessor
from src.feature_extraction import load_optimized_raw_data
from src.features_articles import ArticleEmbeddingResult, get_path_to_article_features
from src.utils.data_checks import test_feature_customer_avg_embedding, test_feature_customer_avg_embedding_pipeline

In [11]:
transactions_train = load_optimized_raw_data(data_type="transactions", sample="train", subsample=0.05, seed=42)
transactions_valid = load_optimized_raw_data(data_type="transactions", sample="valid", subsample=0.05, seed=42)
transactions_test = load_optimized_raw_data(data_type="transactions", sample="test", subsample=0.05, seed=42)
path_article_embeddings = get_path_to_article_features(feature_type="embedding", subsample=1, seed=42)
results_article_embeddings = ArticleEmbeddingResult.load(path_to_dir=path_article_embeddings)

2025-06-03 11:51:50,405 - src.feature_extraction - INFO - Loading optimized raw data from transactions train 0.05 42
2025-06-03 11:51:50,406 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_train_sample_0.05_42.parquet
2025-06-03 11:51:50,434 - src.feature_extraction - INFO - Loading optimized raw data from transactions valid 0.05 42
2025-06-03 11:51:50,435 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_valid_sample_0.05_42.parquet
2025-06-03 11:51:50,438 - src.feature_extraction - INFO - Loading optimized raw data from transactions test 0.05 42
2025-06-03 11:51:50,439 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_test_sample_0.05_42.parquet
2025-06-03 11:51:50,442 - src.features_articles - INFO - Loading article embedding result from ../data/preprocessed/articles_embedding/full
2025-06-03 11:51:50,551 - src.features_articles - INFO - Loaded text embeddings from ../data/preprocessed/articles_embedding/full/

In [12]:
transactions = pd.concat([transactions_train, transactions_valid, transactions_test], ignore_index=True)

In [15]:
# Test metadata of pipeline
test_feature_customer_avg_embedding_pipeline(
    results_customer_dynamic_feature,
    transactions,
    customer_dynamic_feature_config,
)

Checks passed


In [16]:
# Test accuracy of avg embeddings
for customer_id in results_customer_dynamic_feature.data.customer_id.unique()[:5]:
    test_feature_customer_avg_embedding(
        results_customer_dynamic_feature,
        transactions,
        results_article_embeddings,
        customer_dynamic_feature_config,
        customer_id,
    )
    print("=" * 80)

Customer 1296218836199721 has 10 weeks: [ 95  96  97  98  99 100 101 102 103 104]


,t_dat,customer_id,article_id,price,sales_channel_id,week_num


No previous transactions for customer 1296218836199721 and week 95


,t_dat,customer_id,article_id,price,sales_channel_id,week_num


No previous transactions for customer 1296218836199721 and week 96


...

Calculating avg price and embeddings for customer 4380278954532948 and week 102
Checks for customer 4380278954532948 and week 102 passed


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
1499799,2020-08-03,4380278954532948,833499003,0.022864,2,97
1499800,2020-08-03,4380278954532948,833530003,0.015237,2,97


Calculating avg price and embeddings for customer 4380278954532948 and week 103
Checks for customer 4380278954532948 and week 103 passed


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
1499799,2020-08-03,4380278954532948,833499003,0.022864,2,97
1499800,2020-08-03,4380278954532948,833530003,0.015237,2,97


Calculating avg price and embeddings for customer 4380278954532948 and week 104
Checks for customer 4380278954532948 and week 104 passed
